In [1]:
import os

output_folder = r"C:/Users/Dell/internship/week 2/model_outputs"
os.makedirs(output_folder, exist_ok=True)
print("Model outputs folder:", output_folder)


Model outputs folder: C:/Users/Dell/internship/week 2/model_outputs


In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_dir = r"D:/X-ray_dataset/chest_xray/train"
val_dir   = r"D:/X-ray_dataset/chest_xray/val"
test_dir  = r"D:/X-ray_dataset/chest_xray/test"

IMG_SIZE = (224, 224)
BATCH_SIZE = 32

# Training generator with augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary'
)


Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.


In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

def build_custom_cnn(input_shape=(224,224,3)):
    model = Sequential([
        Conv2D(32, (3,3), activation='relu', input_shape=input_shape),
        MaxPooling2D(2,2),
        Conv2D(64, (3,3), activation='relu'),
        MaxPooling2D(2,2),
        Conv2D(128, (3,3), activation='relu'),
        MaxPooling2D(2,2),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(learning_rate=0.0001),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

custom_cnn = build_custom_cnn()
custom_cnn.summary()


C:\Users\Dell\anaconda3new\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 52, 52, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 86528)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │    11,075,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,169,089 (42.61 MB)

 Trainable params: 11,169,089 (42.61 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# -----------------------------
# All-in-One Notebook: Train + Evaluate Models
# -----------------------------

import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, roc_curve
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import VGG16, VGG19, ResNet50, MobileNetV2, DenseNet121

# -----------------------------
# 1️⃣ Paths & Parameters
# -----------------------------
train_dir = r"D:/X-ray_dataset/chest_xray/train"
val_dir   = r"D:/X-ray_dataset/chest_xray/val"
test_dir  = r"D:/X-ray_dataset/chest_xray/test"

output_folder = r"C:/Users/Dell/internship/week 2/model_outputs"
os.makedirs(output_folder, exist_ok=True)

IMG_SIZE = (224,224)
BATCH_SIZE = 32
EPOCHS = 2

# -----------------------------
# 2️⃣ Data Generators
# -----------------------------
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True
)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_gen = train_datagen.flow_from_directory(train_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='binary')
val_gen   = val_datagen.flow_from_directory(val_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='binary')
test_gen  = test_datagen.flow_from_directory(test_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='binary', shuffle=False)

# -----------------------------
# 3️⃣ Custom CNN
# -----------------------------
def build_custom_cnn(input_shape=(224,224,3)):
    model = Sequential([
        Conv2D(32,(3,3),activation='relu', input_shape=input_shape),
        MaxPooling2D(2,2),
        Conv2D(64,(3,3),activation='relu'),
        MaxPooling2D(2,2),
        Conv2D(128,(3,3),activation='relu'),
        MaxPooling2D(2,2),
        Flatten(),
        Dense(128,activation='relu'),
        Dropout(0.5),
        Dense(1,activation='sigmoid')
    ])
    model.compile(optimizer=Adam(1e-4), loss='binary_crossentropy', metrics=['accuracy'])
    return model

custom_cnn = build_custom_cnn()
ckpt_path = os.path.join(output_folder, "Custom_CNN.h5")
custom_cnn.fit(train_gen, validation_data=val_gen, epochs=EPOCHS, callbacks=[ModelCheckpoint(ckpt_path, save_best_only=True)])

# -----------------------------
# 4️⃣ Pretrained Models
# -----------------------------
def build_pretrained_model(base_model):
    base_model.trainable = False
    x = GlobalAveragePooling2D()(base_model.output)
    x = Dense(128,activation='relu')(x)
    out = Dense(1, activation='sigmoid')(x)
    model = Model(inputs=base_model.input, outputs=out)
    model.compile(optimizer=Adam(1e-4), loss='binary_crossentropy', metrics=['accuracy'])
    return model

pretrained_models = {
    "VGG16": VGG16(weights='imagenet', include_top=False, input_shape=(224,224,3)),
    "VGG19": VGG19(weights='imagenet', include_top=False, input_shape=(224,224,3)),
    "ResNet50": ResNet50(weights='imagenet', include_top=False, input_shape=(224,224,3)),
    "MobileNetV2": MobileNetV2(weights='imagenet', include_top=False, input_shape=(224,224,3)),
    "DenseNet121": DenseNet121(weights='imagenet', include_top=False, input_shape=(224,224,3))
}

for name, base in pretrained_models.items():
    print(f"\nTraining {name}...")
    model = build_pretrained_model(base)
    ckpt_path = os.path.join(output_folder, f"{name}.h5")
    model.fit(train_gen, validation_data=val_gen, epochs=EPOCHS, callbacks=[ModelCheckpoint(ckpt_path, save_best_only=True)])

# -----------------------------
# 5️⃣ Evaluate Models
# -----------------------------
model_files = [f for f in os.listdir(output_folder) if f.endswith(".h5")]
models_dict = {os.path.splitext(f)[0]: os.path.join(output_folder, f) for f in model_files}

results = {}
plt.figure(figsize=(8,6))  # combined ROC

for name, path in models_dict.items():
    print(f"\nEvaluating {name}...")
    model = load_model(path)
    y_prob = model.predict(test_gen, verbose=0).reshape(-1)
    y_pred = (y_prob > 0.5).astype(int)
    y_true = test_gen.classes

    # Metrics
    acc = np.mean(y_pred == y_true)
    precision = classification_report(y_true, y_pred, output_dict=True)['1']['precision']
    recall = classification_report(y_true, y_pred, output_dict=True)['1']['recall']
    f1 = classification_report(y_true, y_pred, output_dict=True)['1']['f1-score']
    try:
        roc_auc = roc_auc_score(y_true, y_prob)
    except:
        roc_auc = np.nan

    results[name] = {"Accuracy": acc, "Precision": precision, "Recall": recall, "F1 Score": f1, "ROC-AUC": roc_auc}

    # Confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(5,4))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title(f"{name} - Confusion Matrix")
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.show()

    # ROC curve
    fpr, tpr, _ = roc_curve(y_true, y_prob)
    plt.plot(fpr, tpr, label=f"{name} (AUC={roc_auc:.2f})")

plt.plot([0,1],[0,1],'k--')
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curves - All Models")
plt.legend(loc='lower right')
plt.grid(True)
plt.show()

# -----------------------------
# 6️⃣ Summary Table
# -----------------------------
summary_df = pd.DataFrame(results).T
display(summary_df)

summary_md = summary_df.to_markdown()
with open(os.path.join(output_folder, "model_comparison.md"), "w") as f:
    f.write(summary_md)

fig, ax = plt.subplots(figsize=(10,4))
summary_df[['Accuracy','F1 Score']].plot(kind='bar', ax=ax)
plt.title("Model Comparison - Accuracy & F1 Score")
plt.ylabel("Score")
plt.xticks(rotation=45)
plt.grid(axis='y')
plt.tight_layout()
plt.savefig(os.path.join(output_folder, "model_comparison.png"))
plt.show()


Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


C:\Users\Dell\anaconda3new\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/2
163/163 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 0.7506 - loss: 0.5144   

163/163 ━━━━━━━━━━━━━━━━━━━━ 797s 5s/step - accuracy: 0.8064 - loss: 0.4129 - val_accuracy: 0.6250 - val_loss: 0.6615
Epoch 2/2
163/163 ━━━━━━━━━━━━━━━━━━━━ 0s 25s/step - accuracy: 0.5134 - loss: 0.7843     

163/163 ━━━━━━━━━━━━━━━━━━━━ 4128s 25s/step - accuracy: 0.6603 - loss: 0.6138 - val_accuracy: 0.5000 - val_loss: 0.7715
Epoch 2/2
163/163 ━━━━━━━━━━━━━━━━━━━━ 0s 14s/step - accuracy: 0.7451 - loss: 0.4727   

163/163 ━━━━━━━━━━━━━━━━━━━━ 2357s 14s/step - accuracy: 0.7473 - loss: 0.4515 - val_accuracy: 0.5000 - val_loss: 0.7102

Training VGG19...
Epoch 1/2
163/163 ━━━━━━━━━━━━━━━━━━━━ 0s 27s/step - accuracy: 0.7492 - loss: 0.5437     

163/163 ━━━━━━━━━━━━━━━━━━━━ 4521s 28s/step - accuracy: 0.7429 - loss: 0.5186 - val_accuracy: 0.5000 - val_loss: 0.7044
Epoch 2/2
163/163 ━━━━━━━━━━━━━━━━━━━━ 0s 17s/step - accuracy: 0.7566 - loss: 0.4430     

163/163 ━━━━━━━━━━━━━━━━━━━━ 2872s 17s/step - accuracy: 0.7713 - loss: 0.4230 - val_accuracy: 0.5625 - val_loss: 0.6475

Training ResNet50...
Epoch 1/2
163/163 ━━━━━━━━━━━━━━━━━━━━ 0s 117s/step - accuracy: 0.7571 - loss: 0.5320     

163/163 ━━━━━━━━━━━━━━━━━━━━ 18922s 117s/step - accuracy: 0.7429 - loss: 0.5423 - val_accuracy: 0.5000 - val_loss: 0.7934

Training MobileNetV2...
Epoch 1/2
